In [1]:
from pyspark.sql import SparkSession

MAX_MEMORY="5g"
spark = SparkSession.builder.appName("sparkml_text_fare_prediction")\
                .config("spark.executor.memory",MAX_MEMORY)\
                .config("spark.driver.memory",MAX_MEMORY)\
                .getOrCreate()

22/05/23 17:13:42 WARN Utils: Your hostname, ihyeonmin-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.30.1.11 instead (on interface en0)
22/05/23 17:13:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/23 17:13:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
trip_files = "/Users/ihyeonmin/Desktop/study/data-engineering/01-spark/data/trips/*"

In [3]:
trips_df = spark.read.parquet(f"file:///{trip_files}",inferSchema=True,header=True)

In [4]:
trips_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [5]:
trips_df.createOrReplaceTempView("trips")

In [6]:
query="""
SELECT
    passenger_count,
    PULocationID as pickup_location_id,
    DOLocationID as dropoff_location_id,
    trip_distance,
    HOUR(tpep_pickup_datetime) as pickup_time,
    DATE_FORMAT(TO_DATE(tpep_pickup_datetime),'EEEE') as day_of_week,
    total_amount
FROM
    trips
WHERE
    total_amount<5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_pickup_datetime) < '2021-08-01'
"""

data_df = spark.sql(query)
data_df.createOrReplaceTempView("data")

In [7]:
data_df.show()

+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|day_of_week|total_amount|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|            1.0|                79|                116|          8.4|          9|   Thursday|       35.15|
|            1.0|                75|                236|          0.9|          9|   Thursday|         8.8|
|            1.0|               236|                168|          3.4|          9|   Thursday|        15.3|
|            1.0|               238|                152|         1.96|          9|   Thursday|       13.39|
|            1.0|               142|                238|         0.77|          9|   Thursday|        9.54|
|            1.0|               238|                244|         3.65|          9|   Thursday|       15.36|
|            0.0|           

In [8]:
data_df.printSchema()

root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: long (nullable = true)
 |-- dropoff_location_id: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [9]:
train_df,test_df=data_df.randomSplit([0.8,0.2],seed=1)

In [10]:
data_dir = "/Users/ihyeonmin/Desktop/study/data-engineering/01-spark/data/trips/"

In [13]:
train_df.write.format("parquet").save(f"{data_dir}/train/")
test_df.write.format("parquet").save(f"{data_dir}/test/")


In [14]:
train_df = spark.read.parquet(f"{data_dir}/train/")
test_df = spark.read.parquet(f"{data_dir}/test/")

In [15]:
train_df.printSchema()

root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: long (nullable = true)
 |-- dropoff_location_id: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [16]:
# one hot encoding
# ex) Wednesday -> 4 -> [0,0,0,1,0,0,0]

In [41]:
from pyspark.ml.feature import OneHotEncoder,StringIndexer

cat_feats = [
    "pickup_location_id",
    "dropoff_location_id",
    "day_of_week"
    ]
stages=[]

for c in cat_feats:
    cat_indexer = StringIndexer(inputCol=c,outputCol=c+"_idx").setHandleInvalid("keep")
    onehot_encoder = OneHotEncoder(inputCols=[cat_indexer.getOutputCol()],outputCols=[c+"_onehot"])
    stages += [cat_indexer,onehot_encoder]
print(stages)

[StringIndexer_67bb4c82fd36, OneHotEncoder_a20914cce73a, StringIndexer_73ea154ec26d, OneHotEncoder_b05896f16c04, StringIndexer_bdd88e07fa33, OneHotEncoder_0f9325a30f2f]


In [42]:
from pyspark.ml.feature import VectorAssembler,StandardScaler

num_feats = [
    "passenger_count",
    "trip_distance",
    "total_amount"
]

for n in num_feats:
    num_assembler = VectorAssembler(inputCols=[n],outputCol=n+"_vector")
    num_scaler = StandardScaler(inputCol=num_assembler.getOutputCol(),outputCol=n+"_scaled")
    stages+=[num_assembler,num_scaler]

In [43]:
assembler_inputs = [c + "_onehot" for c in cat_feats] + [n+"_scaled" for n in num_feats]
assembler = VectorAssembler(inputCols=assembler_inputs,outputCol="feature_vector")
stages += [assembler]

In [44]:
from pyspark.ml import Pipeline

transform_stages=stages
pipeline = Pipeline(stages=transform_stages)
fitted_transformer = pipeline.fit(train_df)

In [45]:
vtrain_df = fitted_transformer.transform(train_df)

In [46]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    maxIter=5,
    solver="normal",
    labelCol="total_amount",
    featuresCol="feature_vector"
)

In [47]:
vtrain_df.printSchema()

root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: long (nullable = true)
 |-- dropoff_location_id: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_location_id_idx: double (nullable = false)
 |-- pickup_location_id_onehot: vector (nullable = true)
 |-- dropoff_location_id_idx: double (nullable = false)
 |-- dropoff_location_id_onehot: vector (nullable = true)
 |-- day_of_week_idx: double (nullable = false)
 |-- day_of_week_onehot: vector (nullable = true)
 |-- passenger_count_vector: vector (nullable = true)
 |-- passenger_count_scaled: vector (nullable = true)
 |-- trip_distance_vector: vector (nullable = true)
 |-- trip_distance_scaled: vector (nullable = true)
 |-- total_amount_vector: vector (nullable = true)
 |-- total_amount_scaled: vector (nullable = true)
 |-- feature_vector: vector (nullable

In [48]:
model = lr.fit(vtrain_df)

22/05/23 17:50:59 WARN Instrumentation: [545fb715] regParam is zero, which might cause numerical instability and overfitting.
22/05/23 17:51:12 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/05/23 17:51:12 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/05/23 17:51:19 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
22/05/23 17:51:19 WARN Instrumentation: [545fb715] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
22/05/23 17:51:19 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/05/23 17:51:19 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [50]:
vtest_df = fitted_transformer.transform(test_df)

In [53]:
predictions=model.transform(vtest_df)

In [54]:
predictions.cache()

DataFrame[passenger_count: double, pickup_location_id: bigint, dropoff_location_id: bigint, trip_distance: double, pickup_time: int, day_of_week: string, total_amount: double, pickup_location_id_idx: double, pickup_location_id_onehot: vector, dropoff_location_id_idx: double, dropoff_location_id_onehot: vector, day_of_week_idx: double, day_of_week_onehot: vector, passenger_count_vector: vector, passenger_count_scaled: vector, trip_distance_vector: vector, trip_distance_scaled: vector, total_amount_vector: vector, total_amount_scaled: vector, feature_vector: vector, prediction: double]

In [56]:
predictions.select(["trip_distance","day_of_week","total_amount","prediction"]).show()

+-------------+-----------+------------+------------------+
|trip_distance|day_of_week|total_amount|        prediction|
+-------------+-----------+------------+------------------+
|         13.9|   Saturday|         4.3|29.726438662882963|
|          3.3|     Sunday|         5.8| 8.146875403484225|
|          5.5|     Sunday|       26.75|26.898970528166526|
|          7.9|   Saturday|        30.3|31.777490808363186|
|          0.4|  Wednesday|         7.8| 8.004213711438736|
|          1.0|   Saturday|        11.8|11.705071313777228|
|          1.3|   Saturday|       12.85|12.785841915273393|
|          9.4|   Saturday|        39.8| 40.14312612707506|
|          1.5|  Wednesday|        13.3|13.287095594306242|
|          4.7|     Sunday|       26.15|26.102917853535473|
|          0.6|   Saturday|       11.75|11.327761593278833|
|          1.1|    Tuesday|        11.8|11.722369584818194|
|          3.7|     Friday|       24.35|24.127020715583363|
|          2.4|   Saturday|       14.75|

In [57]:
model.summary.rootMeanSquaredError

1.3337457020031904

In [58]:
model.summary.r2

0.9892934761685851

In [59]:
spark.stop()